# BDA - Project

## 1. Problem description

## 2. Data gathering and storage

## 3. Data description

**Connection with the DB**

In [1]:
import json
import pymongo
from pymongo import MongoClient

client = MongoClient('fpsds.synology.me', 27017, username='mongoadmin', password='bda')

# let's get the list of DBs already defined
print(client.list_database_names())

# Get the DB we want to use
tweets = client['tweets6'].tweets

['admin', 'config', 'local', 'test', 'test_tweepy', 'tweets1', 'tweets2', 'tweets3', 'tweets4', 'tweets5', 'tweets6', 'vaccine_no_since']


Show an example of data sample

In [27]:
tweets.find_one({})

{'_id': 1334789658442665985,
 'created_at': 'Fri Dec 04 09:20:28 +0000 2020',
 'full_text': 'RT @21WIRE: UPDATE: Despite concerns about safety &amp; trial design, countries are already spending billions of dollars in taxpayer funds to p…',
 'display_text_range': [0, 144],
 'entities': {'hashtags': [],
  'symbols': [],
  'user_mentions': [{'screen_name': '21WIRE',
    'name': 'Patrick Henningsen',
    'id': 16936686,
    'id_str': '16936686',
    'indices': [3, 10]}],
  'urls': []},
 'metadata': {'iso_language_code': 'en', 'result_type': 'recent'},
 'geo': None,
 'coordinates': None,
 'place': None,
 'retweet_count': 25,
 'favorite_count': 0}

**Count the total number of tweets on the database and the number of tweets with geolocation**

In [16]:
print("Total number of tweets:", tweets.count_documents({}))

Total number of tweets: 68822


In [26]:
count_with_geo = tweets.count_documents({'$or': [
                                                {'geo': {'$ne': None}},
                                                {'coordinates': {'$ne': None}},
                                                {'place': {'$ne': None}}
                                        ]},{})
print("Number of tweets with geolocation:", count_with_geo)

Number of tweets with geolocation: 931


**Top 25 vaccine related hashtags**

In [37]:
from bson.son import SON

pipeline = [
        {'$unwind': "$entities.hashtags" },
        {'$group': {'_id': '$entities.hashtags.text', 'count': {'$sum': 1}}},
        {"$sort": SON([("count", pymongo.DESCENDING)])},
        {'$limit': 25}
    ]

res = tweets.aggregate(pipeline)
for sample in res:
    print(sample)

{'_id': 'vaccine', 'count': 42747}
{'_id': 'COVID19', 'count': 15302}
{'_id': 'Vaccine', 'count': 9458}
{'_id': 'coronavirus', 'count': 3461}
{'_id': 'COVID', 'count': 3160}
{'_id': 'Covid', 'count': 3068}
{'_id': 'Pfizer', 'count': 2682}
{'_id': 'Covid19', 'count': 2489}
{'_id': 'CovidVaccine', 'count': 2459}
{'_id': 'patents', 'count': 1854}
{'_id': 'COVIDー19', 'count': 1818}
{'_id': 'covid19', 'count': 1412}
{'_id': 'AstraZeneca', 'count': 1315}
{'_id': 'vaccination', 'count': 1207}
{'_id': 'covid', 'count': 1167}
{'_id': 'UK', 'count': 1123}
{'_id': 'BillGates', 'count': 962}
{'_id': 'pandemic', 'count': 794}
{'_id': 'Vaccination', 'count': 789}
{'_id': 'cdc', 'count': 783}
{'_id': 'Moderna', 'count': 777}
{'_id': 'vaccines', 'count': 766}
{'_id': 'GatesFoundation', 'count': 736}
{'_id': 'UN', 'count': 721}
{'_id': 'GAVI', 'count': 718}
